# SECTION 1: Setup and Installation

In [ ]:
# Install required packages
!pip install -q torch torchvision
!pip install -q opencv-python-headless
!pip install -q pillow
!pip install -q matplotlib
!pip install -q seaborn
!pip install -q Flask
!pip install -q fastapi uvicorn
!pip install -q gradio  # For HuggingFace deployment


In [ ]:
# Clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -q -r requirements.txt


Cloning into 'yolov5'...
remote: Enumerating objects: 17739, done.
remote: Counting objects: 100% (96/96), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 17739 (delta 57), reused 31 (delta 31), pack-reused 17643 (from 4)
Receiving objects: 100% (17739/17739), 17.11 MiB | 24.00 MiB/s, done.
Resolving deltas: 100% (12044/12044), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.0 MB/s eta 0:00:00


# Importing Libraries

In [ ]:
import torch
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import os
from pathlib import Path
import json
from IPython.display import display, HTML
import seaborn as sns

print("✓ All packages installed successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

✓ All packages installed successfully!
PyTorch version: 2.8.0+cu126
CUDA available: False


# SECTION 2: Download Pre-trained YOLOv5 Model

In [ ]:
# Using YOLOv5x (most accurate) for best results
# You can also use yolov5s (fastest), yolov5m, yolov5l based on speed needs
MODEL_TYPE = 'yolov5x'  # Change to 'yolov5s' for faster inference

print(f"Loading {MODEL_TYPE} model...")
model = torch.hub.load('ultralytics/yolov5', MODEL_TYPE, pretrained=True)

# Configure model for better detection
model.conf = 0.4  # Confidence threshold
model.iou = 0.45  # NMS IOU threshold
model.classes = None  # Detect all classes, we'll filter for animals

print("✓ Model loaded successfully!")

Loading yolov5x model...


/usr/local/lib/python3.12/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


YOLOv5 🚀 2025-11-18 Python-3.12.12 torch-2.8.0+cu126 CPU

100%|██████████| 166M/166M [00:00<00:00, 259MB/s]

Fusing layers... 
YOLOv5x summary: 444 layers, 86705005 parameters, 0 gradients, 205.5 GFLOPs
Adding AutoShape... 


✓ Model loaded successfully!


# SECTION 3: Animal Class Filtering

In [ ]:
 # COCO dataset animal classes (YOLOv5 is trained on COCO)
ANIMAL_CLASSES = {
    14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep',
    19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe'
}

# Extended animal detection including wildlife
WILDLIFE_CLASSES = list(ANIMAL_CLASSES.keys())

def filter_animal_detections(results):
    """Filter detections to include only animals"""
    df = results.pandas().xyxy[0]
    animal_df = df[df['class'].isin(WILDLIFE_CLASSES)]
    return animal_df

# SECTION 4: Enhanced Detection with Color & Shape Analysis

In [ ]:
class AnimalDetector:
    def __init__(self, model):
        self.model = model
        self.detection_history = []

    def detect_animals(self, image_path, save_path='output'):
        """Main detection function with enhanced features"""
        # Read image
        img = cv2.imread(str(image_path))
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Run YOLOv5 detection
        results = self.model(img_rgb)

        # Filter for animals only
        animal_df = filter_animal_detections(results)

        # Enhanced visualization with color analysis
        annotated_img = self.annotate_image(img_rgb, animal_df)

        # Color and shape analysis for each detection
        enhanced_results = self.analyze_detections(img_rgb, animal_df)

        # Save results
        os.makedirs(save_path, exist_ok=True)
        output_path = os.path.join(save_path, f"detected_{Path(image_path).name}")
        cv2.imwrite(output_path, cv2.cvtColor(annotated_img, cv2.COLOR_RGB2BGR))

        # Store detection history
        self.detection_history.append({
            'image': image_path,
            'detections': len(animal_df),
            'animals': enhanced_results
        })

        return annotated_img, enhanced_results, output_path

    def annotate_image(self, img, detections):
        """Draw bounding boxes with enhanced styling"""
        img_copy = img.copy()

        # Color palette for different animals
        colors = plt.cm.tab10(np.linspace(0, 1, 10))

        for idx, row in detections.iterrows():
            x1, y1, x2, y2 = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
            conf = row['confidence']
            cls = int(row['class'])
            label = ANIMAL_CLASSES.get(cls, 'animal')

            # Get color for this class
            color = tuple(int(c * 255) for c in colors[cls % 10][:3])

            # Draw thick rectangle
            cv2.rectangle(img_copy, (x1, y1), (x2, y2), color, 3)

            # Add label with background
            label_text = f"{label} {conf:.2f}"
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 0.7
            thickness = 2

            (text_w, text_h), _ = cv2.getTextSize(label_text, font, font_scale, thickness)
            cv2.rectangle(img_copy, (x1, y1-text_h-10), (x1+text_w+10, y1), color, -1)
            cv2.putText(img_copy, label_text, (x1+5, y1-5), font, font_scale, (255,255,255), thickness)

            # Add corner markers for professional look
            corner_len = 20
            cv2.line(img_copy, (x1, y1), (x1+corner_len, y1), color, 4)
            cv2.line(img_copy, (x1, y1), (x1, y1+corner_len), color, 4)
            cv2.line(img_copy, (x2, y1), (x2-corner_len, y1), color, 4)
            cv2.line(img_copy, (x2, y1), (x2, y1+corner_len), color, 4)
            cv2.line(img_copy, (x1, y2), (x1+corner_len, y2), color, 4)
            cv2.line(img_copy, (x1, y2), (x1, y2-corner_len), color, 4)
            cv2.line(img_copy, (x2, y2), (x2-corner_len, y2), color, 4)
            cv2.line(img_copy, (x2, y2), (x2, y2-corner_len), color, 4)

        return img_copy

    def analyze_detections(self, img, detections):
        """Enhanced analysis: color distribution and shape metrics"""
        results = []

        for idx, row in detections.iterrows():
            x1, y1, x2, y2 = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
            cls = int(row['class'])
            conf = row['confidence']

            # Extract ROI
            roi = img[y1:y2, x1:x2]

            # Color analysis
            color_info = self.analyze_color(roi)

            # Shape metrics
            shape_info = self.analyze_shape(x1, y1, x2, y2)

            results.append({
                'animal': ANIMAL_CLASSES.get(cls, 'animal'),
                'confidence': float(conf),
                'bbox': [x1, y1, x2, y2],
                'color_analysis': color_info,
                'shape_metrics': shape_info
            })

        return results

    def analyze_color(self, roi):
        """Analyze dominant colors in detected region"""
        if roi.size == 0:
            return {'dominant_color': 'N/A', 'brightness': 0}

        # Calculate average color
        avg_color = roi.mean(axis=(0,1))

        # Determine dominant color name
        r, g, b = avg_color
        dominant = 'red' if r > g and r > b else 'green' if g > r and g > b else 'blue'

        # Calculate brightness
        brightness = np.mean(roi)

        return {
            'dominant_color': dominant,
            'rgb_values': [int(r), int(g), int(b)],
            'brightness': float(brightness)
        }

    def analyze_shape(self, x1, y1, x2, y2):
        """Calculate shape metrics"""
        width = x2 - x1
        height = y2 - y1
        area = width * height
        aspect_ratio = width / height if height > 0 else 0

        return {
            'width': width,
            'height': height,
            'area': area,
            'aspect_ratio': round(aspect_ratio, 2)
        }

    def create_visualization_dashboard(self, results, save_path='output'):
        """Create comprehensive visualization dashboard"""
        if not results:
            print("No animals detected!")
            return

        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        fig.suptitle('Animal Detection Analysis Dashboard', fontsize=16, fontweight='bold')

        # 1. Detection counts
        animal_counts = {}
        for r in results:
            animal = r['animal']
            animal_counts[animal] = animal_counts.get(animal, 0) + 1

        axes[0, 0].bar(animal_counts.keys(), animal_counts.values(), color='skyblue')
        axes[0, 0].set_title('Animal Detection Counts')
        axes[0, 0].set_xlabel('Animal Type')
        axes[0, 0].set_ylabel('Count')
        axes[0, 0].tick_params(axis='x', rotation=45)

        # 2. Confidence scores
        confidences = [r['confidence'] for r in results]
        animals = [r['animal'] for r in results]
        axes[0, 1].barh(animals, confidences, color='lightgreen')
        axes[0, 1].set_title('Detection Confidence Scores')
        axes[0, 1].set_xlabel('Confidence')
        axes[0, 1].set_xlim([0, 1])

        # 3. Size distribution
        areas = [r['shape_metrics']['area'] for r in results]
        axes[1, 0].hist(areas, bins=10, color='coral', edgecolor='black')
        axes[1, 0].set_title('Animal Size Distribution')
        axes[1, 0].set_xlabel('Bounding Box Area (pixels²)')
        axes[1, 0].set_ylabel('Frequency')

        # 4. Aspect ratios
        aspect_ratios = [r['shape_metrics']['aspect_ratio'] for r in results]
        axes[1, 1].scatter(range(len(aspect_ratios)), aspect_ratios,
                          c=confidences, cmap='viridis', s=100, alpha=0.7)
        axes[1, 1].set_title('Aspect Ratio Analysis')
        axes[1, 1].set_xlabel('Detection Index')
        axes[1, 1].set_ylabel('Aspect Ratio (W/H)')
        axes[1, 1].axhline(y=1, color='r', linestyle='--', alpha=0.5)

        plt.tight_layout()
        dashboard_path = os.path.join(save_path, 'analysis_dashboard.png')
        plt.savefig(dashboard_path, dpi=300, bbox_inches='tight')
        plt.show()

        print(f"✓ Dashboard saved to: {dashboard_path}")


# SECTION 5: Download Sample Dataset (Wildlife Images)

In [ ]:

# Download sample images for testing
!mkdir -p sample_images

# Using wget to download sample wildlife images
print("Downloading sample wildlife images...")

# Sample URLs (you can replace with your own dataset)
sample_urls = [
    "https://images.unsplash.com/photo-1564349683136-77e08dba1ef7",  # Tiger
    "https://images.unsplash.com/photo-1535591273668-578e31182c4f",  # Elephant
    "https://images.unsplash.com/photo-1549366021-9f761d450615",  # Dog
]

# Note: In practice, you should use a proper dataset like:
# - COCO animals subset
# - Open Images Dataset (animals)
# - Animals-10 dataset from Kaggle

print("For best results, upload your own animal images to 'sample_images/' folder")


For best results, upload your own animal images to 'sample_images/' folder


# SECTION 6: Run Detection on Sample Images

In [ ]:
detector = AnimalDetector(model)

# Process all images in sample_images folder
image_folder = 'sample_images'
output_folder = 'detection_results'

if os.path.exists(image_folder):
    image_files = [f for f in os.listdir(image_folder)
                   if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    if image_files:
        print(f"\nProcessing {len(image_files)} images...\n")
        all_results = []

        for img_file in image_files:
            img_path = os.path.join(image_folder, img_file)
            print(f"Processing: {img_file}")

            annotated, results, output_path = detector.detect_animals(img_path, output_folder)
            all_results.extend(results)

            # Display result
            plt.figure(figsize=(12, 8))
            plt.imshow(annotated)
            plt.axis('off')
            plt.title(f'Detection Results: {img_file}')
            plt.show()

            # Print detailed results
            print(f"  → Found {len(results)} animal(s)")
            for i, r in enumerate(results, 1):
                print(f"    {i}. {r['animal'].upper()}: {r['confidence']:.2%} confidence")
                print(f"       Color: {r['color_analysis']['dominant_color']}")
                print(f"       Size: {r['shape_metrics']['width']}x{r['shape_metrics']['height']}px")
            print()

        # Create overall dashboard
        if all_results:
            detector.create_visualization_dashboard(all_results, output_folder)

            # Save results to JSON
            json_path = os.path.join(output_folder, 'detection_results.json')
            with open(json_path, 'w') as f:
                json.dump(all_results, f, indent=2)
            print(f"✓ Results saved to: {json_path}")
    else:
        print("No images found in sample_images folder!")
else:
    print("Please create 'sample_images' folder and add images!")


No images found in sample_images folder!


In [ ]:
import urllib.request

# Create sample_images folder
os.makedirs('sample_images', exist_ok=True)

# Sample animal images (free to use)
sample_images = {
    'dog.jpg': 'https://images.unsplash.com/photo-1583511655857-d19b40a7a54e?w=800',
    'cat.jpg': 'https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=800',
    'elephant.jpg': 'https://images.unsplash.com/photo-1564760055775-d63b17a55c44?w=800',
    'horse.jpg': 'https://images.unsplash.com/photo-1553284965-83fd3e82fa5a?w=800',
    'bird.jpg': 'https://images.unsplash.com/photo-1444464666168-49d633b86797?w=800'
}

print("Downloading sample images...")
for filename, url in sample_images.items():
    try:
        urllib.request.urlretrieve(url, f'sample_images/{filename}')
        print(f"✓ Downloaded {filename}")
    except Exception as e:
        print(f"✗ Failed to download {filename}: {e}")

print("\n✓ Sample images ready!")

✓ Downloaded dog.jpg
✓ Downloaded cat.jpg
✓ Downloaded elephant.jpg
✓ Downloaded horse.jpg
✓ Downloaded bird.jpg

✓ Sample images ready!


In [ ]:
detector = AnimalDetector(model)

# Process all images in sample_images folder
image_folder = 'sample_images'
output_folder = 'detection_results'

if os.path.exists(image_folder):
    image_files = [f for f in os.listdir(image_folder)
                   if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    if image_files:
        print(f"\nProcessing {len(image_files)} images...\n")
        all_results = []

        for img_file in image_files:
            img_path = os.path.join(image_folder, img_file)
            print(f"Processing: {img_file}")

            annotated, results, output_path = detector.detect_animals(img_path, output_folder)
            all_results.extend(results)

            # Display result
            plt.figure(figsize=(12, 8))
            plt.imshow(annotated)
            plt.axis('off')
            plt.title(f'Detection Results: {img_file}')
            plt.show()

            # Print detailed results
            print(f"  → Found {len(results)} animal(s)")
            for i, r in enumerate(results, 1):
                print(f"    {i}. {r['animal'].upper()}: {r['confidence']:.2%} confidence")
                print(f"       Color: {r['color_analysis']['dominant_color']}")
                print(f"       Size: {r['shape_metrics']['width']}x{r['shape_metrics']['height']}px")
            print()

        # Create overall dashboard
        if all_results:
            detector.create_visualization_dashboard(all_results, output_folder)

            # Save results to JSON
            json_path = os.path.join(output_folder, 'detection_results.json')
            with open(json_path, 'w') as f:
                json.dump(all_results, f, indent=2)
            print(f"✓ Results saved to: {json_path}")
    else:
        print("No images found in sample_images folder!")
else:
    print("Please create 'sample_images' folder and add images!")



Processing 5 images...

Processing: cat.jpg


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


  → Found 1 animal(s)
    1. CAT: 75.56% confidence
       Color: green
       Size: 465x484px

Processing: dog.jpg


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


  → Found 1 animal(s)
    1. DOG: 94.49% confidence
       Color: green
       Size: 204x349px

Processing: horse.jpg


...

  → Found 1 animal(s)
    1. HORSE: 95.93% confidence
       Color: blue
       Size: 464x332px

Processing: elephant.jpg


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


  → Found 2 animal(s)
    1. ELEPHANT: 95.80% confidence
       Color: red
       Size: 164x201px
    2. ELEPHANT: 94.94% confidence
       Color: red
       Size: 232x163px

Processing: bird.jpg


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


  → Found 1 animal(s)
    1. BIRD: 94.33% confidence
       Color: green
       Size: 291x342px

✓ Dashboard saved to: detection_results/analysis_dashboard.png
✓ Results saved to: detection_results/detection_results.json


SECTION 7: Real-time Webcam Detection (Optional)

In [ ]:
def detect_from_webcam(duration=30):
    """Run detection on webcam feed"""
    from google.colab.patches import cv2_imshow

    cap = cv2.VideoCapture(0)

    print(f"Starting webcam detection for {duration} seconds...")
    start_time = cv2.getTickCount()

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Convert BGR to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Detect
        results = model(frame_rgb)
        animal_df = filter_animal_detections(results)

        # Annotate
        annotated = detector.annotate_image(frame_rgb, animal_df)

        # Show
        cv2_imshow(cv2.cvtColor(annotated, cv2.COLOR_RGB2BGR))

        # Check duration
        elapsed = (cv2.getTickCount() - start_time) / cv2.getTickFrequency()
        if elapsed > duration:
            break

    cap.release()
    print("Webcam detection completed!")

# Uncomment to run webcam detection
# detect_from_webcam(duration=30)

# SECTION 8: Export Model for Deployment

In [ ]:
print("\n" + "="*60)
print("DEPLOYMENT PREPARATION")
print("="*60)

# Save model for deployment
deployment_folder = 'deployment_ready'
os.makedirs(deployment_folder, exist_ok=True)

# Export to ONNX for production
try:
    dummy_input = torch.randn(1, 3, 640, 640)
    torch.onnx.export(model.model, dummy_input,
                     f"{deployment_folder}/animal_detector.onnx",
                     export_params=True,
                     opset_version=11,
                     input_names=['input'],
                     output_names=['output'])
    print("✓ Model exported to ONNX format")
except Exception as e:
    print(f"ONNX export skipped: {e}")

# Save detection history
history_path = os.path.join(deployment_folder, 'detection_history.json')
with open(history_path, 'w') as f:
    json.dump(detector.detection_history, f, indent=2)
print(f"✓ Detection history saved to: {history_path}")

print("\n" + "="*60)
print("PROJECT COMPLETE! 🎉")
print("="*60)
print("\nNext Steps for Deployment:")
print("1. Flask API: Use the saved model and create endpoints")
print("2. FastAPI: Higher performance, async support")
print("3. HuggingFace Spaces: Use Gradio interface (see below)")
print("\nAll outputs saved in 'detection_results/' and 'deployment_ready/'")



DEPLOYMENT PREPARATION


/tmp/ipython-input-3006701166.py:12: DeprecationWarning: You are using the legacy TorchScript-based ONNX export. Starting in PyTorch 2.9, the new torch.export-based ONNX exporter will be the default. To switch now, set dynamo=True in torch.onnx.export. This new exporter supports features like exporting LLMs with DynamicCache. We encourage you to try it and share feedback to help improve the experience. Learn more about the new export logic: https://pytorch.org/docs/stable/onnx_dynamo.html. For exporting control flow: https://pytorch.org/tutorials/beginner/onnx/export_control_flow_model_to_onnx_tutorial.html.
  torch.onnx.export(model.model, dummy_input,
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:695: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  

ONNX export skipped: Module onnx is not installed!
✓ Detection history saved to: deployment_ready/detection_history.json

PROJECT COMPLETE! 🎉

Next Steps for Deployment:
1. Flask API: Use the saved model and create endpoints
2. FastAPI: Higher performance, async support
3. HuggingFace Spaces: Use Gradio interface (see below)

All outputs saved in 'detection_results/' and 'deployment_ready/'


# SECTION 9: Gradio Interface for HuggingFace Deployment

In [ ]:
import gradio as gr

def gradio_detect(image):
    """Gradio inference function"""
    if image is None:
        return None, "Please upload an image"

    # Convert to RGB if needed
    if len(image.shape) == 2:
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

    # Run detection
    results = model(image)
    animal_df = filter_animal_detections(results)

    # Annotate
    annotated = detector.annotate_image(image, animal_df)

    # Create results text
    results_text = f"🎯 Detected {len(animal_df)} animal(s):\n\n"
    for idx, row in animal_df.iterrows():
        animal = ANIMAL_CLASSES.get(int(row['class']), 'animal')
        conf = row['confidence']
        results_text += f"• {animal.upper()}: {conf:.1%} confidence\n"

    return annotated, results_text

# Create Gradio interface
demo = gr.Interface(
    fn=gradio_detect,
    inputs=gr.Image(label="Upload Animal Image"),
    outputs=[
        gr.Image(label="Detection Results"),
        gr.Textbox(label="Detected Animals", lines=5)
    ],
    title="🐾 AVISHKAR 1.0 - Animal Detection System",
    description="Upload an image to detect and analyze animals using YOLOv5",
    examples=[],
    theme=gr.themes.Soft()
)

# Launch interface
print("\nLaunching Gradio interface...")
demo.launch(share=True, debug=True)

print("\n✨ Copy the public URL above to share your demo!")
print("📤 Deploy to HuggingFace Spaces: https://huggingface.co/spaces")


Launching Gradio interface...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://f7def4a7bc38851744.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


In [ ]:
# CHECK MODEL LOCATION AND SAVE IT PROPERLY
print("🔍 Checking Model Location...")

# Your current model is loaded from torch.hub and exists only in memory
print(f"Model type: {type(model)}")
print(f"Model device: {next(model.parameters()).device}")

# The model was downloaded to torch hub cache, but let's save it locally
MODEL_SAVE_PATH = '/content/animal_detection_model.pt'

print("\n💾 Saving model to local file...")
try:
    # Save the entire model
    torch.save(model, MODEL_SAVE_PATH)
    print(f"✅ Model saved to: {MODEL_SAVE_PATH}")

    # Also save just the state dict (lighter)
    STATE_DICT_PATH = '/content/animal_detection_state_dict.pt'
    torch.save(model.state_dict(), STATE_DICT_PATH)
    print(f"✅ State dict saved to: {STATE_DICT_PATH}")

except Exception as e:
    print(f"❌ Error saving model: {e}")

# Check if files were created
if os.path.exists(MODEL_SAVE_PATH):
    file_size = os.path.getsize(MODEL_SAVE_PATH) / (1024 * 1024)  # MB
    print(f"📦 Model file size: {file_size:.2f} MB")
else:
    print("❌ Model file was not created")

# Let's also check the torch hub cache location
import torch.hub as hub
print(f"\n🏠 Torch hub cache directory: {hub.get_dir()}")

# List files in the hub directory to see cached models
hub_dir = hub.get_dir()
if os.path.exists(hub_dir):
    print(f"\n📁 Contents of hub directory:")
    for item in os.listdir(hub_dir):
        item_path = os.path.join(hub_dir, item)
        if os.path.isdir(item_path):
            print(f"  📂 {item}")
            # Check ultralytics_yolov5_master specifically
            if 'ultralytics' in item.lower():
                yolov5_path = os.path.join(item_path, 'yolov5x.pt')
                if os.path.exists(yolov5_path):
                    print(f"    🎯 YOLOv5 model found: {yolov5_path}")

In [ ]:
# PROPER WAY TO SAVE AND LOAD YOLOv5 MODELS
print("\n🚀 PROPER MODEL MANAGEMENT FOR YOLOv5")

def save_yolov5_model_properly():
    """Proper way to save YOLOv5 model for deployment"""

    # Method 1: Save using torch.save (entire model)
    model_path_1 = '/content/yolov5x_complete.pt'
    torch.save(model, model_path_1)
    print(f"✅ Complete model saved: {model_path_1}")

    # Method 2: Save state dict only
    model_path_2 = '/content/yolov5x_state_dict.pt'
    torch.save({
        'model_state_dict': model.state_dict(),
        'model_config': {
            'type': 'yolov5x',
            'classes': ANIMAL_CLASSES,
            'confidence': 0.4,
            'iou': 0.45
        }
    }, model_path_2)
    print(f"✅ State dict + config saved: {model_path_2}")

    # Method 3: Export to ONNX for production
    try:
        # Create dummy input
        dummy_input = torch.randn(1, 3, 640, 640, device=next(model.parameters()).device)

        # Export to ONNX
        onnx_path = '/content/yolov5x_animal_detector.onnx'
        torch.onnx.export(
            model,
            dummy_input,
            onnx_path,
            input_names=['input'],
            output_names=['output'],
            dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}},
            opset_version=12
        )
        print(f"✅ ONNX model exported: {onnx_path}")
    except Exception as e:
        print(f"⚠️ ONNX export failed: {e}")

    return model_path_1, model_path_2

# Save models properly
saved_paths = save_yolov5_model_properly()

# Verify the saved models can be loaded
print("\n🔍 Verifying saved models...")
for path in saved_paths:
    if os.path.exists(path):
        try:
            # Load the model
            loaded_model = torch.load(path)
            print(f"✅ Successfully loaded: {path}")

            # Test the loaded model
            test_image = torch.randn(1, 3, 640, 640)
            with torch.no_grad():
                output = loaded_model(test_image)
                print(f"   Test output shape: {output[0].shape if isinstance(output, tuple) else output.shape}")

        except Exception as e:
            print(f"❌ Failed to load {path}: {e}")
    else:
        print(f"❌ File not found: {path}")